# Introduction to Semi GAN

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import scale


import numpy as np
import pandas as pd

In [2]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv1D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Lambda
from keras.layers import Activation
from matplotlib import pyplot
from keras import backend
from keras.models import Sequential

Using TensorFlow backend.


In [4]:
df = pd.read_csv("https://github.com/mpourhoma/CS4661/raw/master/Heart_s.csv")
df.head()

,Age,Gender,ChestPain,RestBP,Chol,RestECG,MaxHR,Oldpeak,Thal,AHD
0,63,f,typical,145,233,2,150,2.3,fixed,No
1,67,f,asymptomatic,160,286,2,108,1.5,normal,Yes
2,67,f,asymptomatic,120,229,2,129,2.6,reversable,Yes
3,37,f,nonanginal,130,250,0,187,3.5,normal,No
4,41,m,nontypical,130,204,2,172,1.4,normal,No


## All Features:

#### All Features and OneHotEncoding:

In [5]:
f_col_all = ["Age", "Gender" , "ChestPain" , "RestBP" , "Chol" , "RestECG" , "MaxHR" , "Oldpeak" , "Thal"]

X = df[f_col_all]  # all features
#print(X.head())

X_OneHotEncoded = pd.get_dummies(X)  # all features and OneHotEncoded
f_col_OHE = list(X_OneHotEncoded.columns.values)
y = df["AHD"].apply(lambda x: 1 if x== "Yes" else 0 )  # Labels
print(X_OneHotEncoded.head())


   Age  RestBP  Chol  ...  Thal_fixed  Thal_normal  Thal_reversable
0   63     145   233  ...           1            0                0
1   67     160   286  ...           0            1                0
2   67     120   229  ...           0            0                1
3   37     130   250  ...           0            1                0
4   41     130   204  ...           0            1                0

[5 rows x 15 columns]


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_OneHotEncoded, y, test_size=0.25, random_state=6)
print(X_test.shape)
print(y_test.shape)

(76, 15)
(76,)


In [6]:
my_decisiontree = DecisionTreeClassifier(random_state=5)
my_decisiontree.fit(X_train, y_train)
y_predict_dt = my_decisiontree.predict(X_test)
score_dt = accuracy_score(y_test, y_predict_dt)
print(score_dt)


0.7368421052631579


In [7]:
my_logreg = LogisticRegression()
my_logreg.fit(X_train, y_train)
y_predict_lr = my_logreg.predict(X_test)
score_lr = accuracy_score(y_test, y_predict_lr)
print(score_lr)


0.7631578947368421


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [8]:
my_knn = KNeighborsClassifier(n_neighbors=5)
my_knn.fit(X_train, y_train)
y_predict_knn = my_knn.predict(X_test)
score_knn = accuracy_score(y_test, y_predict_knn)
print(score_knn)

0.6973684210526315


In [8]:
model = Sequential()
input_size = 15
hidden_neurons = 100
out_size = 1

# second layer: hidden layer:
model.add(Dense(hidden_neurons, input_dim = input_size))  # Nuerons
model.add(Activation('sigmoid')) # Activation

# -----------------------------------------
# third layer: output layer:
model.add(Dense(out_size, input_dim = hidden_neurons))  # Nuerons
model.add(Activation('sigmoid')) # Activation

model.compile(loss='binary_crossentropy',
              metrics=['accuracy'],
              optimizer='adam')

fitted_model = model.fit(X_train, y_train, validation_split=0.33, batch_size=32, epochs=100, verbose=1)
_, acc = model.evaluate(X_test, y_test, verbose=1)
acc

Train on 150 samples, validate on 75 samples
Epoch 1/100
150/150 [==============================] - 0s 2ms/step - loss: 0.6893 - acc: 0.5600 - val_loss: 0.6727 - val_acc: 0.5333
Epoch 2/100
150/150 [==============================] - 0s 217us/step - loss: 0.6675 - acc: 0.6133 - val_loss: 0.6707 - val_acc: 0.5333
Epoch 3/100
150/150 [==============================] - 0s 221us/step - loss: 0.6613 - acc: 0.6267 - val_loss: 0.6731 - val_acc: 0.5467
Epoch 4/100
150/150 [==============================] - 0s 207us/step - loss: 0.6567 - acc: 0.6267 - val_loss: 0.6698 - val_acc: 0.5867
Epoch 5/100
150/150 [==============================] - 0s 203us/step - loss: 0.6501 - acc: 0.6200 - val_loss: 0.6717 - val_acc: 0.5333
Epoch 6/100
150/150 [==============================] - 0s 222us/step - loss: 0.6441 - acc: 0.6667 - val_loss: 0.6681 - val_acc: 0.5733
Epoch 7/100
150/150 [==============================] - 0s 203us/step - loss: 0.6409 - acc: 0.6467 - val_loss: 0.6657 - val_acc: 0.6133
Epoch 8/100


0.7368421021260714

# **GAN**

In [0]:
# custom activation function
def custom_activation(output):
	logexpsum = backend.sum(backend.exp(output), axis=-1, keepdims=True)
	result = logexpsum / (logexpsum + 1.0)
	return result

# define the standalone supervised and unsupervised discriminator models
def define_discriminator(in_shape=(15,), n_classes=2):
  # image input
  in_image = Input(shape=in_shape)
  # downsample
  fe = Dense(units=100, activation='sigmoid')(in_image)
  fe = LeakyReLU(alpha=0.2)(fe)
  # downsample
  fe = Dense(units=100, activation='sigmoid')(fe)
  fe = LeakyReLU(alpha=0.2)(fe)
  # downsample
  fe = Dense(units=100, activation='sigmoid')(fe)
  fe = LeakyReLU(alpha=0.2)(fe)
  # dropout
  fe = Dropout(0.4)(fe)
  # output layer nodes
  fe = Dense(n_classes)(fe)
  # supervised output
  c_out_layer = Activation('softmax')(fe)
  # define and compile supervised discriminator model
  c_model = Model(in_image, c_out_layer)
  c_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])
  # unsupervised output
  d_out_layer = Lambda(custom_activation)(fe)
  # define and compile unsupervised discriminator model
  d_model = Model(in_image, d_out_layer)
  d_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
  return d_model, c_model

# define the standalone generator model
def define_generator(latent_dim, n_outputs=15):
	model = Sequential()
	model.add(Dense(200, activation='sigmoid', kernel_initializer='he_uniform', input_dim=latent_dim))
	model.add(Dense(100, activation='sigmoid'))
	model.add(Dense(n_outputs, activation='relu'))
	return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect image output from generator as input to discriminator
	gan_output = d_model(g_model.output)
	# define gan model as taking noise and outputting a classification
	model = Model(g_model.input, gan_output)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# load the images
def load_real_samples(X,y):
	print(X.shape, y.shape)
	return [X, y]

# select a supervised subset of the dataset, ensures classes are balanced
def select_supervised_samples(dataset, n_samples=100, n_classes=2):
  X, y = dataset
  rand = randint(0,1000)
  n_per_class = int(n_samples / n_classes)
  return X.sample(n=n_per_class,random_state=rand), y.sample(n=n_per_class,random_state=rand)

# select real samples
def generate_real_samples(dataset, n_samples):
	# split into images and labels
  features, labels = dataset
	# choose random instances
  rand = randint(0, 1000)
	# select features and labels
  X = features.sample(n=n_samples,random_state=rand)
  labels = labels.sample(n=n_samples,random_state=rand)
	# generate class labels
  y = ones((n_samples, 1))
  return [X, labels], y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	z_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = z_input.reshape(n_samples, latent_dim)
	return z_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	z_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	images = generator.predict(z_input)
	# create class labels
	y = zeros((n_samples, 1))
	return images, y

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, c_model, latent_dim, dataset, acc_list, n_samples=100):
	# prepare fake examples
	X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
	# scale from [-1,1] to [0,1]
	X = (X + 1) / 2.0
	# evaluate the classifier model
	X, y = dataset
	_, acc = c_model.evaluate(X, y, verbose=0)
	acc_list.append(acc)
	print('Classifier Accuracy: %.3f%%' % (acc * 100))
	# save the generator model
	filename2 = 'g_model_%04d.h5' % (step+1)
	g_model.save(filename2)
	# save the classifier model
	filename3 = 'c_model_%04d.h5' % (step+1)
	c_model.save(filename3)
	print('>Saved:  %s and %s' % (filename2, filename3))

# train the generator and discriminator
def train(g_model, d_model, c_model, gan_model, dataset, latent_dim, acc_list, n_epochs=5000, n_batch=100):
	# select supervised dataset
	X_sup, y_sup = select_supervised_samples(dataset)
	print(X_sup.shape, y_sup.shape)
	# calculate the number of batches per training epoch
	bat_per_epo = int(dataset[0].shape[0] / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# calculate the size of half a batch of samples
	half_batch = int(n_batch / 2)
	print('n_epochs=%d, n_batch=%d, 1/2=%d, b/e=%d, steps=%d' % (n_epochs, n_batch, half_batch, bat_per_epo, n_steps))
	# manually enumerate epochs
	for i in range(n_steps):
		# update supervised discriminator (c)
		[Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], half_batch)
		c_loss, c_acc = c_model.train_on_batch(Xsup_real, ysup_real)
		# update unsupervised discriminator (d)
		[X_real, _], y_real = generate_real_samples(dataset, half_batch)
		d_loss1 = d_model.train_on_batch(X_real, y_real)
		X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		d_loss2 = d_model.train_on_batch(X_fake, y_fake)
		# update generator (g)
		X_gan, y_gan = generate_latent_points(latent_dim, n_batch), ones((n_batch, 1))
		g_loss = gan_model.train_on_batch(X_gan, y_gan)
		# summarize loss on this batch
		print('>%d, c[%.3f,%.0f], d[%.3f,%.3f], g[%.3f]' % (i+1, c_loss, c_acc*100, d_loss1, d_loss2, g_loss))
		# evaluate the model performance every so often
		if (i+1) % (500) == 0:
			summarize_performance(i, g_model, c_model, latent_dim, dataset, acc_list)



In [13]:
#accuracy list for each epochs
acc_list = []
# size of the latent space
latent_dim = 100
# create the discriminator models
d_model, c_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples(X_train,y_train)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
(225, 15) (225,)


In [0]:
# train model
train(g_model, d_model, c_model, gan_model, dataset, latent_dim, acc_list)

(50, 15) (50,)
n_epochs=5000, n_batch=100, 1/2=50, b/e=2, steps=10000


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, c[0.851,40], d[0.380,1.235], g[0.376]
>2, c[0.970,38], d[0.379,1.162], g[0.392]
>3, c[0.843,46], d[0.358,1.132], g[0.403]
>4, c[0.727,62], d[0.369,1.165], g[0.431]
>5, c[0.823,50], d[0.419,1.111], g[0.405]
>6, c[0.974,34], d[0.397,1.176], g[0.440]
>7, c[0.707,60], d[0.415,1.117], g[0.422]
>8, c[0.728,54], d[0.418,1.048], g[0.421]
>9, c[0.783,50], d[0.462,1.071], g[0.443]
>10, c[0.820,52], d[0.447,1.118], g[0.454]
>11, c[0.789,40], d[0.483,1.077], g[0.446]
>12, c[0.761,48], d[0.430,1.095], g[0.492]
>13, c[0.757,60], d[0.447,1.036], g[0.492]
>14, c[0.709,62], d[0.482,1.035], g[0.505]
>15, c[0.787,46], d[0.458,1.029], g[0.459]
>16, c[0.899,38], d[0.441,1.021], g[0.487]
>17, c[0.760,54], d[0.459,0.996], g[0.486]
>18, c[0.934,38], d[0.479,1.057], g[0.481]
>19, c[0.759,50], d[0.451,1.028], g[0.476]
>20, c[0.747,54], d[0.461,0.953], g[0.510]
>21, c[0.756,54], d[0.454,1.037], g[0.524]
>22, c[0.643,58], d[0.496,0.966], g[0.550]
>23, c[0.897,38], d[0.453,0.916], g[0.498]
>24, c[0.723,54], d[

In [0]:
from keras.models import load_model
for x in range(500,10000,500):
  # load the model
  model = load_model("c_model_%04d.h5" % x)
  # evaluate the model
  _, train_acc = model.evaluate(X_train, y_train, verbose=0)
  print(x,"epochs")
  
  print('Train Accuracy: %.3f%%' % (train_acc * 100))
  _, test_acc = model.evaluate(X_test, y_test, verbose=0)
  print('Test Accuracy: %.3f%%' % (test_acc * 100))

500 epochs
Train Accuracy: 70.222%
Test Accuracy: 69.737%
1000 epochs
Train Accuracy: 79.111%
Test Accuracy: 76.316%
1500 epochs
Train Accuracy: 80.000%
Test Accuracy: 73.684%
2000 epochs
Train Accuracy: 80.889%
Test Accuracy: 73.684%
2500 epochs
Train Accuracy: 78.667%
Test Accuracy: 80.263%
3000 epochs
Train Accuracy: 77.333%
Test Accuracy: 76.316%
3500 epochs
Train Accuracy: 78.222%
Test Accuracy: 76.316%
4000 epochs
Train Accuracy: 78.667%
Test Accuracy: 77.632%
4500 epochs
Train Accuracy: 79.111%
Test Accuracy: 77.632%
5000 epochs
Train Accuracy: 78.667%
Test Accuracy: 77.632%
5500 epochs
Train Accuracy: 79.111%
Test Accuracy: 75.000%
6000 epochs
Train Accuracy: 78.667%
Test Accuracy: 77.632%
6500 epochs
Train Accuracy: 79.556%
Test Accuracy: 72.368%
7000 epochs
Train Accuracy: 80.444%
Test Accuracy: 68.421%
7500 epochs
Train Accuracy: 79.556%
Test Accuracy: 71.053%
8000 epochs
Train Accuracy: 80.000%
Test Accuracy: 71.053%
8500 epochs
Train Accuracy: 80.444%
Test Accuracy: 69.737